In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization, Conv2D, DepthwiseConv2D, Dropout, Flatten, Dense, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from PIL import Image
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers


# 定義模型保存路徑
model_save_path = './best_model/model_35_long_150_100/'
if not os.path.exists(model_save_path):
    os.makedirs(model_save_path)

# 創建ModelCheckpoint回調，以在每個epoch後保存驗證損失最小的模型
mcp_save = ModelCheckpoint(filepath=model_save_path, save_best_only=True, monitor='val_sparse_categorical_accuracy', mode='max', verbose=1)

weight_decay = 1e-4

class Preact_resnet(keras.Model):
    def __init__(self, input_shape = (224,224,3) ):
        super(Preact_resnet, self).__init__()
        self.layer_1 = tf.keras.models.Sequential([
                        tf.keras.layers.Conv2D(32, 3, padding='same', strides=2,
                           activation=None, input_shape=input_shape, kernel_regularizer=regularizers.l2(weight_decay)),
                        BatchNormalization(),
                        tf.keras.layers.Activation('relu'),
                        tf.keras.layers.Dropout(0.1),
                        ])
        ## stage 1
        self.block_1 = tf.keras.models.Sequential([

            tf.keras.layers.DepthwiseConv2D(3, padding='same', activation=None, strides=1, kernel_regularizer=regularizers.l2(weight_decay)),
            BatchNormalization(),
            tf.keras.layers.Conv2D(32, 1, padding='same', activation=None, strides=1, kernel_regularizer=regularizers.l2(weight_decay)),
            BatchNormalization(),
            tf.keras.layers.Activation('relu'),


            tf.keras.layers.DepthwiseConv2D(3, padding='same', activation=None, strides=1, kernel_regularizer=regularizers.l2(weight_decay)),
            BatchNormalization(),
            tf.keras.layers.Conv2D(32, 1, padding='same', activation=None, strides=1, kernel_regularizer=regularizers.l2(weight_decay)),
            BatchNormalization(),
            tf.keras.layers.Activation('relu'),
            tf.keras.layers.Dropout(0.1),

            ])
        # shortcut_1 is identity
        self.block_2 = tf.keras.models.Sequential([
            tf.keras.layers.DepthwiseConv2D(3, padding='same', activation=None, strides=1, kernel_regularizer=regularizers.l2(weight_decay)),
            BatchNormalization(),
            tf.keras.layers.Conv2D(32, 1, padding='same', activation=None, strides=1, kernel_regularizer=regularizers.l2(weight_decay)),
            BatchNormalization(),
            tf.keras.layers.Activation('relu'),


            tf.keras.layers.DepthwiseConv2D(3, padding='same', activation=None, strides=1, kernel_regularizer=regularizers.l2(weight_decay)),
            BatchNormalization(),
            tf.keras.layers.Conv2D(32, 1, padding='same', activation=None, strides=1, kernel_regularizer=regularizers.l2(weight_decay)),
            BatchNormalization(),
            tf.keras.layers.Activation('relu'),
            tf.keras.layers.Dropout(0.1),
            ])
        # shortcut_2 is identity

        ## stage 2
        self.block_3 = tf.keras.models.Sequential([
            tf.keras.layers.DepthwiseConv2D(3, padding='same', activation=None, strides=2, kernel_regularizer=regularizers.l2(weight_decay)),
            BatchNormalization(),
            tf.keras.layers.Conv2D(64, 1, padding='same', activation=None, strides=1, kernel_regularizer=regularizers.l2(weight_decay)),

            BatchNormalization(),
            tf.keras.layers.Activation('relu'),
            tf.keras.layers.DepthwiseConv2D(3, padding='same', activation=None, strides=1, kernel_regularizer=regularizers.l2(weight_decay)),
            BatchNormalization(),
            tf.keras.layers.Conv2D(64, 1, padding='same', activation=None, strides=1, kernel_regularizer=regularizers.l2(weight_decay)),

            BatchNormalization(),
            tf.keras.layers.Activation('relu'),
            tf.keras.layers.DepthwiseConv2D(3, padding='same', activation=None, strides=1, kernel_regularizer=regularizers.l2(weight_decay)),
            BatchNormalization(),
            tf.keras.layers.Conv2D(64, 1, padding='same', activation=None, strides=1, kernel_regularizer=regularizers.l2(weight_decay)),
            BatchNormalization(),
            tf.keras.layers.Activation('relu'),
            tf.keras.layers.Dropout(0.1),
            ])
        self.shortcut_3 = tf.keras.layers.Conv2D(64, 1, activation=None, strides=2, kernel_regularizer=regularizers.l2(weight_decay))

        self.block_4 = tf.keras.models.Sequential([
            tf.keras.layers.DepthwiseConv2D(3, padding='same', activation=None, strides=1, kernel_regularizer=regularizers.l2(weight_decay)),
            BatchNormalization(),
            tf.keras.layers.Conv2D(64, 1, padding='same', activation=None, strides=1, kernel_regularizer=regularizers.l2(weight_decay)),

            BatchNormalization(),
            tf.keras.layers.Activation('relu'),
            tf.keras.layers.DepthwiseConv2D(3, padding='same', activation=None, strides=1, kernel_regularizer=regularizers.l2(weight_decay)),
            BatchNormalization(),
            tf.keras.layers.Conv2D(64, 1, padding='same', activation=None, strides=1, kernel_regularizer=regularizers.l2(weight_decay)),

            BatchNormalization(),
            tf.keras.layers.Activation('relu'),
            tf.keras.layers.DepthwiseConv2D(3, padding='same', activation=None, strides=1, kernel_regularizer=regularizers.l2(weight_decay)),
            BatchNormalization(),
            tf.keras.layers.Conv2D(64, 1, padding='same', activation=None, strides=1, kernel_regularizer=regularizers.l2(weight_decay)),
            BatchNormalization(),
            tf.keras.layers.Activation('relu'),
            tf.keras.layers.Dropout(0.1),
            ])
        # shortcut_4 is identity

        ## stage 3
        self.block_5 = tf.keras.models.Sequential([
            tf.keras.layers.DepthwiseConv2D(3, padding='same', activation=None, strides=2, kernel_regularizer=regularizers.l2(weight_decay)),
            BatchNormalization(),
            tf.keras.layers.Conv2D(128, 1, padding='same', activation=None, strides=1, kernel_regularizer=regularizers.l2(weight_decay)),

            BatchNormalization(),
            tf.keras.layers.Activation('relu'),
            tf.keras.layers.DepthwiseConv2D(3, padding='same', activation=None, strides=1, kernel_regularizer=regularizers.l2(weight_decay)),
            BatchNormalization(),
            tf.keras.layers.Conv2D(128, 1, padding='same', activation=None, strides=1, kernel_regularizer=regularizers.l2(weight_decay)),

            BatchNormalization(),
            tf.keras.layers.Activation('relu'),
            tf.keras.layers.DepthwiseConv2D(3, padding='same', activation=None, strides=1, kernel_regularizer=regularizers.l2(weight_decay)),
            BatchNormalization(),
            tf.keras.layers.Conv2D(128, 1, padding='same', activation=None, strides=1, kernel_regularizer=regularizers.l2(weight_decay)),
            BatchNormalization(),
            tf.keras.layers.Activation('relu'),
            tf.keras.layers.Dropout(0.1),
            ])
        self.shortcut_5 = tf.keras.layers.Conv2D(128, 1, activation=None, strides=2, kernel_regularizer=regularizers.l2(weight_decay))

        self.block_6 = tf.keras.models.Sequential([
            tf.keras.layers.DepthwiseConv2D(3, padding='same', activation=None, strides=1, kernel_regularizer=regularizers.l2(weight_decay)),
            BatchNormalization(),
            tf.keras.layers.Conv2D(128, 1, padding='same', activation=None, strides=1, kernel_regularizer=regularizers.l2(weight_decay)),

            BatchNormalization(),
            tf.keras.layers.Activation('relu'),
            tf.keras.layers.DepthwiseConv2D(3, padding='same', activation=None, strides=1, kernel_regularizer=regularizers.l2(weight_decay)),
            BatchNormalization(),
            tf.keras.layers.Conv2D(128, 1, padding='same', activation=None, strides=1, kernel_regularizer=regularizers.l2(weight_decay)),

            BatchNormalization(),
            tf.keras.layers.Activation('relu'),
            tf.keras.layers.DepthwiseConv2D(3, padding='same', activation=None, strides=1, kernel_regularizer=regularizers.l2(weight_decay)),
            BatchNormalization(),
            tf.keras.layers.Conv2D(128, 1, padding='same', activation=None, strides=1, kernel_regularizer=regularizers.l2(weight_decay)),
            BatchNormalization(),
            tf.keras.layers.Activation('relu'),
            tf.keras.layers.Dropout(0.1),
            ])
        # shortcut_4 is identity

        self.out_layer = tf.keras.models.Sequential([

            ## output layer
            tf.keras.layers.AveragePooling2D((4, 4)),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(15, activation=None),
            ])


    def call(self, inputs):
        # inputs = (inputs - 0.4733) * 4 # mean = 0.4733, std = 0.251568723192402

        x = self.layer_1(inputs)

        # stage 1
        x = self.block_1(x) + x
        x = self.block_2(x) + x

        # stage 2
        shortcut = self.shortcut_3(x)
        x = self.block_3(x) + shortcut
        x = self.block_4(x) + x

        # stage 3
        shortcut = self.shortcut_5(x)
        x = self.block_5(x) + shortcut
        x = self.block_6(x) + x

        x = self.out_layer(x)

        return x

# 讀取CSV文件
df = pd.read_csv('/content/drive/MyDrive/archive/dogs.csv')

# 選擇指定類別
selected_breeds = [
    "Airedale", "Beagle", "Bloodhound", "Bluetick", "Chihuahua",
    "Collie", "Dingo", "French Bulldog", "German Shepherd",
    "Malinois", "Newfoundland", "Pekinese", "Pomeranian", "Pug", "Vizsla"
]

# 過濾指定品種
df_selected = df[df['labels'].isin(selected_breeds)]

# 加載圖片
def load_images(df):
    image_list = []
    for idx, row in df.iterrows():
        img = Image.open('/content/drive/MyDrive/archive/' + row['filepaths'])
        img = img.resize((224, 224))  # 使用224x224大小的圖像
        img_array = np.array(img) / 255.0
        image_list.append(img_array)
    return np.array(image_list)

images = load_images(df_selected)

# label邊碼
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(df_selected['labels'])

# 分割訓練集
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


# 數據增強
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False
)

# 實體化模型
model = Preact_resnet(input_shape=(224, 224, 3))

# 定learning_rate
learning_rate = 0.0004

# 定Epoch數量
epochs_num = 10

# 編譯模型
model.compile(optimizer=Adam(learning_rate=learning_rate),  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=[keras.metrics.SparseCategoricalAccuracy()])

# 訓練模型，並將ModelCheckpoint加入到callbacks中
model.fit(datagen.flow(X_train, y_train, batch_size=32),
          epochs=epochs_num,
          validation_data=(X_test, y_test),
          callbacks=[mcp_save])  # 將mcp_save加入到訓練過程中

# 加載最佳模型
best_model = tf.keras.models.load_model(model_save_path, compile=False)

# 結果
val_loss, val_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy with learning rate {learning_rate}: {val_acc}\n")
#0.001=0.52 0.0005=0.55 0.0001=0.32 0.0007=0.46 0.0004=0.57 0.0003=0.49 0.00035=0.46 0.00041=0.52 0.00042=00.579

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
import os

def load_and_preprocess_image(image_path, target_size=(224, 224)):
    img = image.load_img(image_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return tf.keras.applications.mobilenet_v2.preprocess_input(img_array_expanded_dims)

def predict_and_evaluate(model, image_dir):
    correct_predictions = 0
    total_images = 0

    for root, dirs, files in os.walk(image_dir):
        for file in files:
            if file.lower().endswith(('png', 'jpg', 'jpeg')):
                img_path = os.path.join(root, file)
                processed_img = load_and_preprocess_image(img_path)
                prediction = model.predict(processed_img)
                predicted_class = np.argmax(prediction, axis=1)[0]

                total_images += 1

    accuracy = correct_predictions / total_images if total_images > 0 else 0
    print(f"Accuracy: {accuracy:.2f}")

# 加载模型
model_path = './best_model/model_35_long_150_100/'
model = tf.keras.models.load_model(model_path)

# 验证数据集的路径
validation_images_path = '/content/drive/MyDrive/archive/valid'

# 进行预测并评估
predict_and_evaluate(model, validation_images_path)


OSError: No file or directory found at ./best_model/model_35_long_150_100/

In [3]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
import tensorflow as tf

# 指定圖片文件夾路徑
test_images_directory = '/content/drive/MyDrive/archive/Testing set'

# 加載模型
model = load_model('./best_model/model_35_long_150_100/', compile=False)

# 讀取圖片並進行預測
def predict_images_from_directory(directory):
    predicted_labels = []
    file_names = []
    for file in os.listdir(directory):
        file_path = os.path.join(directory, file)
        img = image.load_img(file_path, target_size=(224, 224))
        img_array = image.img_to_array(img)
        img_array_expanded_dims = np.expand_dims(img_array, axis=0)
        img_preprocessed = tf.keras.applications.mobilenet_v2.preprocess_input(img_array_expanded_dims)

        prediction = model.predict(img_preprocessed)
        predicted_class = np.argmax(prediction, axis=1)

        # 儲存文件名和預測結果
        file_names.append(file)
        predicted_labels.append(predicted_class[0])

    return file_names, predicted_labels

# 使用函數進行預測
file_names, predicted_classes = predict_images_from_directory(test_images_directory)

# 創建結果DataFrame
results_df = pd.DataFrame({
    'FileName': file_names,
    'PredictedClass': predicted_classes
})

# 將結果保存為Excel文件
results_df.to_excel('test_data.xlsx', index=False)


OSError: No file or directory found at ./best_model/model_35_long_150_100/